In [1]:
import warnings

import pandas as pd
import numpy as np

from functions.loading import load_data

from functions.merged_dataset_creation import create_preprocessed_dataset
from functions.training_pipeline import training_pipeline
from functions.models import xgboost_model, catboost_model, lgbm_model

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None


## Parameters defintion

In [2]:
path_rawdata = 'data/raw_data/'
path_models = 'models/proprietary_data/'
path_Benchmark = 'Benchmark/'
path_results = 'results/proprietary_data/'
path_plot = path_results +'plot/'
path_intermediary = 'data/intermediary_data/proprietary_data/'
path_plot = 'results/proprietary_data/plot/'
# ,"CF3_log", "CF123_log"
targets = ["CF1_log","CF2_log","CF3_log", "CF123_log"]
models = {
        "xgboost": xgboost_model,
        "catboost": catboost_model,
        "lgbm": lgbm_model,
}
training_parameters = {
    "low":0.01,
    "high":1,
    "extended_features": [
            "Revenue_log",
            "EMP_log",
            "Asset_log",
            "NPPE_log",
            "CapEx_log",
            "Age",
            "CapInten",
            "GMAR",
            "Leverage",
            "Price",
            "FuelIntensity",
            "FiscalYear",
            "ENEConsume_log",
            "ENEProduce_log",
            "INTAN_log",
            "AccuDep_log",
            "COGS_log",
        ],
    "selec_sect":["GICSSubInd", "GICSInd", "GICSGroup"],
    "fill_grp":"",
    "old_pipe":False,  
    "cross_val": False,
}

Summary_Final=[]
Summary_Final_train = []
ensemble =[]
summary_metrics_detailed = pd.DataFrame()
estimated_scopes = []

In [3]:
# Refinitiv_cdp_merged = pd.read_csv(path_rawdata+"Refinitiv_cdp_merged.csv")


## Train and save best models for proprietary data

In [4]:
# Refinitiv_cdp_merged = pd.read_csv(path_rawdata+"Refinitiv_cdp_merged.csv")
# Refinitiv_cdp_merged['CDP_CF1'].fillna(Refinitiv_cdp_merged['CF1'], inplace=True)
# Refinitiv_cdp_merged['CDP_CF2_location'].fillna(Refinitiv_cdp_merged['CF2'], inplace=True)
# Refinitiv_cdp_merged['CDP_CF3'].fillna(Refinitiv_cdp_merged['CF3'], inplace=True)

In [5]:
Refinitiv_data, CarbonPricing, IncomeGroup, FuelIntensity, GICSReclass = load_data(path_rawdata) 
        
preprocessed_dataset = create_preprocessed_dataset(
    Refinitiv_data,
    GICSReclass,
    CarbonPricing,
    IncomeGroup,
    FuelIntensity) 
index_to_nan = preprocessed_dataset[(preprocessed_dataset.CF123.notna()) & ((preprocessed_dataset.CF2.isna()) | (preprocessed_dataset.CF3.isna()))].index
preprocessed_dataset.loc[index_to_nan, "CF123"] = [np.nan for i in range(len(index_to_nan))]

In [6]:
# scope = "CF1"
# nb_std=2
# Refinitiv_data = preprocessed_dataset.copy()

# company_names_list = []
# selected_companies = []

# Refinitiv_data[f"intensity_{scope}"] = Refinitiv_data[scope] / Refinitiv_data["Revenue"]

# for subindustry in Refinitiv_data["GICSName"].unique():
#     subset = Refinitiv_data[Refinitiv_data["GICSName"] == subindustry]

#     std_subind = np.std(subset[f"intensity_{scope}"])
#     mean_subind = np.mean(subset[f"intensity_{scope}"])
#     max_subind = mean_subind + nb_std * std_subind
#     min_subind = mean_subind - nb_std * std_subind

#     condition = (subset[f"intensity_{scope}"] > max_subind) | (subset[f"intensity_{scope}"] < min_subind)

#     if any(condition):
#         selected_companies.extend(
#             subset.loc[condition, ["Name", "FiscalYear"]].apply(lambda x: (x["Name"], x["FiscalYear"]), axis=1)
#         )

# selected_companies_years = selected_companies
# company_names = [company[0] for company in selected_companies_years]

# company_names_list.append(company_names)

In [7]:
# preprocessed_dataset

In [8]:
# test XX : restricted to CF123
targets = ["CF1_log_CF123","CF2_log_CF123","CF3_log_CF123", "CF123_log"]

best_scores, best_stds, summary_global, summary_global_train, summary_metrics_detailed,df_test,df_train, df_test_before_imputation,df_train_before_imputation = training_pipeline(
    name_experiment="restriction_CF123_test_4",
    path_Benchmark=path_Benchmark,
    path_results=path_results,
    path_models=path_models,
    path_intermediary=path_intermediary,
    path_plot = path_plot,
    targets=targets,
    models=models,
    Summary_Final=Summary_Final,
    # Summary_Final_train =Summary_Final_train,
    ensemble=ensemble,
    summary_metrics_detailed=summary_metrics_detailed,
    estimated_scopes = estimated_scopes,
    preprocessed_dataset=preprocessed_dataset,
    training_parameters=training_parameters,
    open_data=False,
    save=False,
)

ValueError: too many values to unpack (expected 6)

In [11]:
from functions.preprocessing import target_preprocessing, custom_train_split
target = targets[0]
(
    X_train,
    y_train,
    X_test,
    y_test,
    df_test,
    df_train,
    df_test_before_imputation,
    df_train_before_imputation,
) = custom_train_split(
    preprocessed_dataset,
    path_Benchmark,
    path_intermediary,
    target,
    low=training_parameters["low"],
    high=training_parameters["high"],
    extended_features=training_parameters["extended_features"],
    selec_sect=training_parameters["selec_sect"],
    fill_grp=training_parameters["fill_grp"],
    old_pipe=training_parameters["old_pipe"],
    open_data=False,
)

In [18]:
df_train_before_imputation.columns.tolist()

['FinalEikonID',
 'Name',
 'CUSIP',
 'ISIN',
 'Ticker',
 'CountryHQ',
 'NAICSSector',
 'NAICSInd',
 'GICSSector',
 'GICSGroup',
 'GICSInd',
 'GICSSubInd',
 'GICSName',
 'FiscalYear',
 'CF1',
 'CF2',
 'CF3',
 'CF12',
 'CFE',
 'EstMethod',
 'CF123',
 'Revenue',
 'EBITDA',
 'EBIT',
 'CapEx',
 'GPPE',
 'NPPE',
 'AccuDep',
 'INTAN',
 'COGS',
 'GMAR',
 'Asset',
 'LTDebt',
 'EMP',
 'ENEProduce',
 'ENEConsume',
 'Region',
 'CO2Law',
 'CO2Scheme',
 'CO2Status',
 'CO2Coverage',
 'StartYear',
 'Status',
 'Price',
 'Area',
 'Year',
 'FuelIntensity',
 'IncomeGroup',
 'SubInd',
 'SubIndName',
 'Recat',
 'Recat2',
 'Age',
 'CapInten',
 'Leverage']

In [47]:
# a = df_train_before_imputation[df_train_before_imputation.FiscalYear>=2011].dropna(subset="CF123")
# a[a.CO2Status=="Implemented"][[ "CO2Status","CO2Coverage", "CO2Law"]].CO2Law.value_counts() # CO2Status #CO2Coverage

Yes    6356
TBD      76
Name: CO2Law, dtype: int64

In [50]:
a = df_train_before_imputation[df_train_before_imputation.FiscalYear>=2011].dropna(subset="CF123")
a["FiscalYear"].value_counts()

2021    1956
2020    1694
2019    1367
2018    1045
2017     848
2016     749
2012     656
2015     644
2013     612
2014     582
2011     574
Name: FiscalYear, dtype: int64

In [83]:
preprocessed_dataset[(preprocessed_dataset.IncomeGroup.isna()) & (preprocessed_dataset.FiscalYear>=2005)].CountryHQ.value_counts()

Guernsey    63
Jersey      48
Name: CountryHQ, dtype: int64

In [76]:
df_train_before_imputation[df_train_before_imputation.IncomeGroup.isna()]

,FinalEikonID,Name,CUSIP,ISIN,Ticker,CountryHQ,NAICSSector,NAICSInd,GICSSector,GICSGroup,...,FuelIntensity,IncomeGroup,SubInd,SubIndName,Recat,Recat2,Age,CapInten,Leverage,IncomeGroup_encoded
1564,SRET.L,Sirius Real Estate Ltd,NaN,GG00B1W3VF54,SRE,Guernsey,53,531120.0,60.0,6010.0,...,379.38,NaN,60102020.0,Real Estate Operating Companies,Real Estate,Real Estate,8.215501,0.097477,0.418723,NaN
4250,SREI.L,Schroder Real Estate Investment Trust Ltd,NaN,GB00B01HM147,SREI,Guernsey,52,525990.0,60.0,6010.0,...,379.38,NaN,60101010.0,Diversified REITs,Real Estate,Real Estate,NaN,NaN,0.300534,NaN
4416,JLEC.L,Jersey Electricity PLC,NaN,JE00B43SP147,JEL,Jersey,22,221122.0,55.0,5510.0,...,379.38,NaN,55101010.0,Electric Utilities,Utilities,Utilities,36.292750,1.852008,0.099126,NaN
4492,SHAN.L,Shanta Gold Ltd,NaN,GB00B0CGR828,SHG,Guernsey,21,212220.0,15.0,1510.0,...,379.38,NaN,15104030.0,Gold,Materials,Materials,17.685941,0.887980,0.016807,NaN
6603,APIA.L,abrdn Property Income Trust Ltd,NaN,GB0033875286,API,Guernsey,53,531120.0,60.0,6010.0,...,379.38,NaN,60101010.0,Diversified REITs,Real Estate,Real Estate,NaN,0.076500,0.210088,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76498,8604.T,Nomura Holdings Inc,NaN,JP3762600009,8604,Japan,52,523150.0,40.0,4020.0,...,419.46,NaN,40203020.0,Investment Banking & Brokerage,Diversified Financials,Diversified Financials,11.592115,0.219840,0.107200,NaN
76499,HSBA.L,HSBC Holdings PLC,NaN,GB0005405286,HSBA,United Kingdom,52,522110.0,40.0,4010.0,...,481.57,NaN,40101010.0,Diversified Banks,Banks,Banks,9.623782,0.503748,0.070308,NaN
76500,ERST.VI,Erste Group Bank AG,NaN,AT0000652011,EBS,Austria,52,522110.0,40.0,4010.0,...,219.31,NaN,40101010.0,Diversified Banks,Banks,Banks,10.962525,0.496699,0.145002,NaN
76501,EURBr.AT,Eurobank Ergasias Services and Holdings SA,NaN,GRS323013003,EUROB,Greece,52,522110.0,40.0,4010.0,...,705.10,NaN,40101010.0,Diversified Banks,Banks,Banks,10.453651,0.555696,0.000000,NaN


In [99]:
# df_train_before_imputation[["IncomeGroup"]]
X.loc[:,['animals']]

,animals
0,low
1,med
2,low
3,high
4,low
5,high


In [93]:
X = pd.DataFrame({'animals':['low','med','low','high','low','high']})
enc = OrdinalEncoder(categories=[['low', 'med', 'high']])
enc.fit_transform(X.loc[:,['animals']])

array([[0.],
       [1.],
       [0.],
       [2.],
       [0.],
       [2.]])

In [101]:
from sklearn.preprocessing import OrdinalEncoder
df_train_before_imputation["IncomeGroup"] = df_train_before_imputation["IncomeGroup"].fillna("H")
income_group_encoder = OrdinalEncoder(categories=[['H', 'UM', 'LM', 'L']])
df_train_before_imputation["IncomeGroup_encoded"] = income_group_encoder.fit_transform(df_train_before_imputation[["IncomeGroup"]])
df_train_before_imputation.IncomeGroup.value_counts()

H     55496
UM     9317
LM     2537
L        22
Name: IncomeGroup, dtype: int64

In [104]:
df_train_before_imputation.IncomeGroup_encoded.value_counts()

0.0    55496
1.0     9317
2.0     2537
3.0       22
Name: IncomeGroup_encoded, dtype: int64

In [67]:
income_group_encoder.categories_ 

[array(['H', 'L', 'LM', 'UM', nan], dtype=object)]

In [66]:
df_train_before_imputation.IncomeGroup_encoded.value_counts()

0.0    53898
3.0     9317
2.0     2537
1.0       22
Name: IncomeGroup_encoded, dtype: int64

In [59]:
a['IncomeGroup'].value_counts()

H     9145
UM    1267
LM     308
Name: IncomeGroup, dtype: int64

In [58]:
X_train['IncomeGroup_encoded'].value_counts()

0.0    9050
3.0    1255
2.0     306
Name: IncomeGroup_encoded, dtype: int64

In [14]:
X_train.columns.tolist()

['Revenue_log',
 'EMP_log',
 'Asset_log',
 'NPPE_log',
 'CapEx_log',
 'Age',
 'CapInten',
 'GMAR',
 'Leverage',
 'Price',
 'FuelIntensity',
 'FiscalYear',
 'ENEConsume_log',
 'ENEProduce_log',
 'INTAN_log',
 'AccuDep_log',
 'COGS_log',
 'FinalCO2Law_encoded',
 'IncomeGroup_encoded',
 'FiscalYear_encoded',
 'GICSSubInd__10101010_0',
 'GICSSubInd__10101020_0',
 'GICSSubInd__10102010_0',
 'GICSSubInd__10102020_0',
 'GICSSubInd__10102030_0',
 'GICSSubInd__10102040_0',
 'GICSSubInd__10102050_0',
 'GICSSubInd__15101010_0',
 'GICSSubInd__15101020_0',
 'GICSSubInd__15101030_0',
 'GICSSubInd__15101040_0',
 'GICSSubInd__15101050_0',
 'GICSSubInd__15102010_0',
 'GICSSubInd__15103010_0',
 'GICSSubInd__15103020_0',
 'GICSSubInd__15104010_0',
 'GICSSubInd__15104020_0',
 'GICSSubInd__15104025_0',
 'GICSSubInd__15104030_0',
 'GICSSubInd__15104040_0',
 'GICSSubInd__15104045_0',
 'GICSSubInd__15104050_0',
 'GICSSubInd__15105010_0',
 'GICSSubInd__15105020_0',
 'GICSSubInd__20101010_0',
 'GICSSubInd__2010

In [6]:
# summary_global.to_csv("summary_perf_merge_test_same_bdd.csv")
# summary_global_train.to_csv("summary_perf_merge_train_same_bdd.csv")


In [ ]:
summary_global

In [ ]:
best_scores